In [1]:
import asyncio


loop = asyncio.get_event_loop()
loop.is_running()

False

In [2]:
import functools


def hello_world():
    print('hello world!')
    

def stop_loop(loop):
    print('stopping loop')
    loop.stop()

loop.call_soon(hello_world)

<Handle hello_world() at <ipython-input-2-63b9d7810e6e>:4>

In [3]:
loop.call_soon(functools.partial(stop_loop, loop))

<Handle stop_loop(<_WindowsSele...e debug=False>)() at <ipython-input-2-63b9d7810e6e>:8>

In [4]:
loop.run_forever()

hello world!
stopping loop


In [5]:
loop.call_later(10, hello_world)
loop.call_later(20, functools.partial(stop_loop, loop))

<TimerHandle when=126201.078 stop_loop(<_WindowsSele...e debug=False>)() at <ipython-input-2-63b9d7810e6e>:8>

In [6]:
loop.run_forever()

hello world!


stopping loop


In [7]:
@asyncio.coroutine
def trivial():
    return 'hello world'

In [9]:
loop.run_until_complete(trivial())

'hello world'

In [12]:
import asyncio
import threading


def run_loop_forever_in_background(loop):
    def thread_fun(l):
        asyncio.set_event_loop(l)
        l.run_forever()
    thread = threading.Thread(target=thread_fun, args=(loop,))
    thread.start()
    return thread

In [13]:
loop = asyncio.get_event_loop()
thread = run_loop_forever_in_background(loop)
loop.is_running()

True

In [1]:
import asyncio
import functools

loop = asyncio.get_event_loop()
loop.call_soon_threadsafe(functools.partial(print, 'hello world'))

<Handle print('hello world')()>

In [2]:
@asyncio.coroutine
def coro_sum(*args):
    answer = 0
    for i in args:
        answer += i
    return answer

In [3]:
loop.run_until_complete(coro_sum(1, 2, 3, 3))

hello world


9

In [8]:
try:
    next(coro_sum(1, 2, 3))
except StopIteration as ex:
    print(ex.value)

6


In [9]:
@asyncio.coroutine
def nested(*args):
    return [i + 1 for i in args]


@asyncio.coroutine
def outer(*args):
    answer = yield from nested(*[i * 2 for i in args])
    return answer

loop = asyncio.get_event_loop()
loop.run_until_complete(outer(2, 3, 4, 5))

[5, 7, 9, 11]

In [12]:
import time


@asyncio.coroutine
def outer():
    time.sleep(3)
    print('hello')
    return 52
    
loop = asyncio.get_event_loop()
loop.run_until_complete(outer())

hello


52

In [13]:
@asyncio.coroutine
def make_tea(var):
    print("now making %s tea" % var)
    return "%s tea" % var


def confirm_tea(future):
    print("the %s is made" % future.result())

task = asyncio.async(make_tea("green"))
task.add_done_callback(confirm_tea)
loop.run_until_complete(task)

now making green tea
the green tea is made


'green tea'

In [2]:
import asyncio

queue = asyncio.Queue()
queue.put_nowait('foo')
queue.qsize()

1

In [3]:
queue.get_nowait()
queue.qsize()

0

In [4]:
loop = asyncio.get_event_loop()
queue = asyncio.Queue()

queue.put_nowait('foo')
loop.run_until_complete(queue.get())

'foo'

In [5]:
queue.put('foo')
queue.qsize()

0

In [6]:
loop.run_until_complete(queue.put('foo'))

In [8]:
queue.get_nowait()

QueueEmpty: 

In [9]:
loop = asyncio.get_event_loop()
queue = asyncio.Queue()

task = asyncio.async(queue.get())
coro = asyncio.wait([task] ,timeout=1)

loop.run_until_complete(coro)

(set(),
 {<Task pending coro=<Queue.get() running at D:\Users\a\Anaconda3\lib\asyncio\queues.py:168> wait_for=<Future pending cb=[Task._wakeup()]>>})

In [11]:
queue.put_nowait('foo')
task.done()

False

In [13]:
import functools


def stop(l, future):
    l.stop()

task.add_done_callback(functools.partial(stop, loop))
loop.run_forever()


In [14]:
task.done()

True

In [15]:
task.result()

'foo'

asyncio 模块提供了一种聚合任务的遍历方法。聚合任务主要归于两个原因。第一个原因是在一组任务中的任何任务完成后采取某些行动。第二个原因是在所在任务都完成后采取某些行动

In [16]:
import asyncio

loop = asyncio.get_event_loop()


@asyncio.coroutine
def make_tea(variety):
    print('now making %s tea' % variety)
    return '%s tea' % variety

meta_task = asyncio.gather(
    make_tea('chamomile'),
    make_tea('green'),
    make_tea('herbal')
)

In [17]:
meta_task.done()

False

In [18]:
loop.run_until_complete(meta_task)

now making herbal tea
now making green tea
now making chamomile tea


['chamomile tea', 'green tea', 'herbal tea']

In [19]:
meta_task.done()

True

In [23]:
meta_task = asyncio.gather(
    make_tea('chamomile'),
    make_tea('green'),
    make_tea('herbal')
)


def mix(future):
    print('mixing the %s together' % ' and '.join(future.result()))

meta_task.add_done_callback(meta_task)
loop.run_until_complete(meta_task)

Exception in callback <_GatheringFuture finished result=['chamomile tea', 'green tea', 'herbal tea']>(<_GatheringFu...'herbal tea']>)
handle: <Handle <_GatheringFuture finished result=['chamomile tea', 'green tea', 'herbal tea']>(<_GatheringFu...'herbal tea']>)>
Traceback (most recent call last):
  File "D:\Users\a\Anaconda3\lib\asyncio\events.py", line 125, in _run
    self._callback(*self._args)
TypeError: '_GatheringFuture' object is not callable


now making green tea
now making chamomile tea
now making herbal tea


['chamomile tea', 'green tea', 'herbal tea']

In [24]:
loop = asyncio.get_event_loop()


@asyncio.coroutine
def make_tea(variety):
    print('now making %s tea' % variety)
    return '%s tea' % variety

coro = asyncio.wait([make_tea('green'), make_tea('herbal')])

loop.run_until_complete(coro)

now making herbal tea
now making green tea


({<Task finished coro=<make_tea() done, defined at D:\Users\a\Anaconda3\lib\asyncio\coroutines.py:204> result='herbal tea'>,
  <Task finished coro=<make_tea() done, defined at D:\Users\a\Anaconda3\lib\asyncio\coroutines.py:204> result='green tea'>},
 set())

In [25]:
coro

<generator object wait at 0x000001346C4FBF10>

In [26]:
loop = asyncio.get_event_loop()
coro = asyncio.wait([asyncio.sleep(5), asyncio.sleep(1)], timeout=3)
loop.run_until_complete(coro)

({<Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>},
 {<Task pending coro=<sleep() running at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:516> wait_for=<Future pending cb=[Task._wakeup()]>>})

In [27]:
coro = asyncio.wait([
    asyncio.sleep(1),
    asyncio.sleep(3),
    asyncio.sleep(2)
], return_when=asyncio.FIRST_COMPLETED)

loop.run_until_complete(coro)

({<Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>},
 {<Task pending coro=<sleep() running at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:516> wait_for=<Future pending cb=[Task._wakeup()]>>,
  <Task pending coro=<sleep() running at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:516> wait_for=<Future pending cb=[Task._wakeup()]>>})

In [28]:
@asyncio.coroutine
def raise_ex_after(seconds):
    yield from asyncio.sleep(seconds)
    raise RuntimeError('raising an exception')

coro = asyncio.wait([asyncio.sleep(1), asyncio.sleep(2), raise_ex_after(3), asyncio.sleep(4)],
                    return_when=asyncio.FIRST_EXCEPTION)

loop.run_until_complete(coro)


({<Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>,
  <Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>,
  <Task finished coro=<raise_ex_after() done, defined at <ipython-input-28-9dc7904612bc>:1> exception=RuntimeError('raising an exception',)>},
 {<Task pending coro=<sleep() running at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:516> wait_for=<Future pending cb=[Task._wakeup()]>>})

In [30]:
coro = asyncio.wait([asyncio.sleep(1), asyncio.sleep(2), asyncio.sleep(4)], return_when=asyncio.FIRST_EXCEPTION)
loop.run_until_complete(coro)

({<Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>,
  <Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>,
  <Task finished coro=<sleep() done, defined at D:\Users\a\Anaconda3\lib\asyncio\tasks.py:502> result=None>},
 set())